<a href="https://colab.research.google.com/github/ivoryRabbit/kakao_arena/blob/master/6_Item2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 07/08 결과

- Song nDCG: 0.176309
- Tag nDCG: 0.478919
- 참고사항
    - tag token 및 title token을 사용
        - song nDCG 약간 상승
        - tag nDCG 매우 상승
    - 아직 song meta embedding 사용하지 않음
    - cold start 모델도 아직 그대로 사용
- future works
    - song meta embedding 사용
        - song이 있으면 song_embedding 평균내서 concatenate
        - song 없고 tag 있으면 걍 이대로
    - cold start 모델을 like_cnt, updt_date, title 사용하여 개선하고 싶음
    - 플레이리스트에 할당된 앨범 개수, 아티스트 개수, 장르 개수 등을 활용하여 CF를 사용할 데이터와 CBF를 사용할 데이터를 분류하는 클러스터 작업 필요
        - 예를 들어, song 개수는 많지만 아티스트 개수가 하나라면?
        - CBF가 필요한 데이터는 song embedding을 활용하여 적용할 예정


### 07/11 결과

- Song nDCG: 0.178314
- Tag nDCG: 0.467752
- 참고사항
    - tag token 및 title token을 사용
    - 플레이리스트 속 곡들의 장르 키워드를 모아 추가로 학습
    - 곡 점수는 올랐는데 태그 점수는 떨어짐

- Song nDCG: 0.178655
- Tag nDCG:	0.478250
- 참고사항
    - tag token 및 title token을 사용
    - 플레이리스트 속 곡들의 장르 코드를 학습
    - 위에서 키워드를 썼을 때보다 점수가 상향된 것으로 보아, 첫번째 실험은 overfitting된 듯

### 07/12 결과

- Song nDCG: 0.177532
- Tag nDCG: 0.471922
- 참고사항
    - tag token 및 title token을 사용
    - 플레이리스트 속 곡들의 장르 코드를 학습
    - 연도를 빼고 결측들은 모두 coldstart로 처리
    - 발매연도나 업데이트연도는 굉장히 중요한걸로....

### 07/13 결과

- Song nDCG: 0.178698
- Tag nDCG: 0.477737
- 참고사항
    - tag token 및 title token을 사용
    - __<u>중요</u>__ valid, test셋을 사용하지 않고 오직 train으로만 학습
    - 플레이리스트 속 곡들의 장르 코드를 학습
    - 연도 다시 넣음

### 07/16 결과

- Song nDCG: 0.178707
- Tag nDCG: 0.477928
- 참고사항
    - tag token 및 title token을 사용
    - __<u>중요</u>__ valid, test셋을 사용하지 않고 오직 train으로만 학습
    - 플레이리스트 속 곡들의 장르 코드를 학습
    - 연도 다시 넣음
    - cosine similarity를 cosmul similarity로 바꾸었는데 성능 살짝 향상
        - [참고](https://www.aclweb.org/anthology/W14-1618.pdf)
    - 하루 빨리 song meta vector 추가하고 싶은데 너무 귀찮

### 07/18 결과

- Song nDCG: 0.180802
- Tag nDCG: 0.481057
- 참고사항
    - tag token 및 title token을 사용
    - __<u>중요</u>__ valid, test셋을 사용하지 않고 오직 train으로만 학습
    - 플레이리스트 속 곡들의 장르 코드를 학습
    - 연도 다시 넣음
    - cosine similarity를 cosmul similarity로 바꾸었는데 성능 살짝 향상
        - [참고](https://www.aclweb.org/anthology/W14-1618.pdf)
    - 하루 빨리 song meta vector 추가하고 싶은데 너무 귀찮
    - dimension 256으로 늘림
        - 점수가 별로 안오르는걸로보아 모델적인 한계가 분명 있음
    - 새로운 coldstart 모델 적용

이번주 주말 예정사항
- 토요일날 song meta vector(a.k.a. content) 마저 작업
- 어떻게 content 사용할지 그동안 생각
- 일요일날 되도록 점수 크게 상향시킬것
- 토요일까지 체력 회복시키기

In [ ]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')

my_path = '/content/notebooks'
try: os.symlink('/content/drive/My Drive/Colab Notebooks/my_env', my_path)
except: print('Already does')
sys.path.insert(0, my_path)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import re, glob
import json
import numpy as np
import pandas as pd

from tqdm.notebook import tqdm

from gensim.models import Word2Vec
from gensim.models.keyedvectors import WordEmbeddingsKeyedVectors

In [ ]:
display(glob.glob('drive/My Drive/kakao_arena/data/*'))
base_path = 'drive/My Drive/kakao_arena/'

['drive/My Drive/kakao_arena/data/val.json',
 'drive/My Drive/kakao_arena/data/test.json',
 'drive/My Drive/kakao_arena/data/genre_gn_all.json',
 'drive/My Drive/kakao_arena/data/train.json',
 'drive/My Drive/kakao_arena/data/song_meta.json',
 'drive/My Drive/kakao_arena/data/val_v1.json',
 'drive/My Drive/kakao_arena/data/test_v1.json',
 'drive/My Drive/kakao_arena/data/train_v1.json',
 'drive/My Drive/kakao_arena/data/genre_v1.json',
 'drive/My Drive/kakao_arena/data/song_meta_v1.json',
 'drive/My Drive/kakao_arena/data/test_v2.json',
 'drive/My Drive/kakao_arena/data/val_v2.json',
 'drive/My Drive/kakao_arena/data/train_v2.json',
 'drive/My Drive/kakao_arena/data/song_meta_v2.json']

In [ ]:
train = pd.read_json(base_path + 'data/train_v2.json')
valid = pd.read_json(base_path + 'data/val_v2.json')
test = pd.read_json(base_path + 'data/test_v2.json')

In [ ]:
song_meta = pd.read_json(base_path + 'data/song_meta_v2.json')
song_meta.head(3)

,song_gnr_dtl_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gnr_basket,artist_name_basket,songs,song_gnr_keywords,like_cnt,popular,song_name_basket,album_name_basket
0,[GN0901],20140512,불후의 명곡 7080 추억의 얄개시대 팝송베스트,2255639,[],feelings,[GN0900],[],0,"[국외, 팝]",0.058775,0.142951,[feeling],"[불후, 명곡, 추억, 시대, 팝송, 베스트]"
1,"[GN1601, GN1606]",20080421,bach partitas nos 2 3 4,376431,[29966],bach partita no 4 in d major bwv 828 ii allemande,[GN1600],[murray perahia],1,"[독주곡, 클래식]",0.000000,0.000000,"[bach, part, major, bwv, man]","[bach, part]"
2,[GN0901],20180518,hit,4698747,[3361],solsbury hill remastered 2002,[GN0900],[peter gabriel],2,"[국외, 팝]",0.000000,0.000000,[remaster],[hit]


In [ ]:
valid_cold = pd.read_json(base_path + 'submissions/valid_cold_2.json')
test_cold = pd.read_json(base_path + 'submissions/test_cold_2.json')

In [ ]:
song_gnr_map = {idx: song_meta.at[idx, 'song_gnr_basket'] for idx in tqdm(song_meta.index)}
song_art_map = {idx: song_meta.at[idx, 'artist_name_basket'] for idx in tqdm(song_meta.index)}

In [ ]:
def trunc_items(df):
    tag_cnts = df.tags.explode().dropna().value_counts()
    tag_sprs = tag_cnts[tag_cnts < 22].index
    up_df = df.assign(tags = lambda x: x.tags.apply(lambda x: [y for y in x if y not in tag_sprs]))

    song_cnts = df.songs.explode().dropna().value_counts()
    song_sprs = song_cnts[song_cnts < 29].index
    up_df = up_df.assign(songs = lambda x: x.songs.apply(lambda x: [y for y in x if y not in song_sprs]))
    return up_df

In [ ]:
#train = trunc_items(train)

In [ ]:
def extend_items(df):
    song_ex = df.songs.explode().dropna()

    gnr_basket = song_ex.map(song_gnr_map)
    gnr_basket = gnr_basket.groupby(level = 0).agg(list)
    gnr_basket = gnr_basket.apply(lambda x: [z for y in x for z in y])
    gnr_basket = gnr_basket.apply(lambda x: list(set(x)))
    gnr_basket = gnr_basket.rename('gnr_basket')
    up_df = df.join(gnr_basket)

    art_basket = song_ex.map(song_art_map)
    art_basket = art_basket.groupby(level = 0).agg(list)
    art_basket = art_basket.apply(lambda x: [z for y in x for z in y])
    art_basket = art_basket.apply(lambda x: list(set(x)))
    art_basket = art_basket.rename('art_basket')
    up_df = up_df.join(art_basket)
    up_df = up_df.fillna('NAN').applymap(lambda x: [] if x == 'NAN' else x)
    return up_df

In [ ]:
train, valid, test = map(extend_items, (train, valid, test))
train.head(3)

,tags,id,plylst_title,songs,like_cnt,updt_date,title_basket,tag_basket,gnr_basket,art_basket
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,20131219,"[여행, 음악]",[록],"[GN1300, GN1100, GN1900, GN1000, GN1400, GN0900]","[james blunt, honeymoon suite, red hot chili p..."
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,20141202,[요즘],"[추억, 회상]","[GN1800, GN1700, GN0500, GN1000, GN0600, GN080...","[어쿠스틱 콜라보, 임주연, 브로콜리너마저, ryu miho, above envy,..."
2,"[까페, 잔잔한]",76951,편하게 잔잔하게 들을 수 있는 곡,"[83116, 276692, 166267, 186301, 354465, 256598...",17,20170828,[잔잔],"[까페, 잔잔]","[GN0300, GN1700, GN0500, GN0400, GN0600, GN080...","[common tale, 김박첼라, 쏠라티, 타코앤제이형, perc%nt, 샘김 (..."


In [ ]:
def make_items(df):
    user_info = df.assign(
        songs = lambda x: x.songs.apply(lambda x: list(map(str, x))),
        updt_date = lambda x: x.updt_date.apply(lambda x: str(x)[:4] + 'y')
    )
    user_info = user_info.assign(
        total_users = lambda x: x.tag_basket + x.songs + x.title_basket + x.gnr_basket + x.art_basket + x.updt_date.apply(lambda x: [x])
    )
    user_info['total_items'] = user_info.total_users.apply(lambda x: list(set(x)))
    return user_info.set_index('id')[['tags', 'songs', 'total_items']]

In [ ]:
train_items, valid_items, test_items = map(make_items, (train, valid, test))
train_items.head(3)

,tags,songs,total_items
id,,,
61281,[락],"[525514, 129701, 383374, 562083, 297861, 13954...","[223955, 50104, 461973, GN1300, 록, GN1100, GN1..."
10532,"[추억, 회상]","[432406, 675945, 497066, 120377, 389529, 24427...","[97749, 추억, 어쿠스틱 콜라보, 383960, GN1700, GN0500, ..."
76951,"[까페, 잔잔한]","[83116, 276692, 166267, 186301, 354465, 256598...","[common tale, 471385, 김박첼라, GN1700, GN0500, 27..."


In [ ]:
def Embedding(Series, dim):
    items = Series.tolist()
    W2V = Word2Vec(items,
                   window = max([len(i) for i in items]), 
                   size = dim,
                   min_count = 3,
                   workers = 4,
                   negative = 5,
                   sg = 1)
    print('finish...')
    return W2V
    
def Embedding_Mapper(df, model):
    ID = []
    vec = []
    for idx in tqdm(df.index):
        N = 0
        temp_vec = 0
        for w in df.at[idx, 'total_items']:
            try: temp_vec += model.wv.get_vector(w)
            except: pass
            else: N += 1
        if type(temp_vec) == int:
            continue
        ID.append(str(idx))
        vec.append(temp_vec / N)
    vec = np.array(vec)

    WEKV = WordEmbeddingsKeyedVectors(vec.shape[1])
    WEKV.add(ID, vec)
    return WEKV

In [ ]:
%%time
#user2vec = Embedding(plylst_info.total_items, 256)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 7.39 µs


In [ ]:
user2vec.save(base_path + 'models/user2vec_200719.model')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
user2vec = Word2Vec.load(base_path + 'models/user2vec_200719.model')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
train_mapped, valid_mapped, test_mapped = map(lambda x: Embedding_Mapper(x, user2vec), (train_items, valid_items, test_items))

In [ ]:
tag_dic = {str(train.at[idx, 'id']): train.at[idx, 'tags'] for idx in tqdm(train.index)}
song_dic = {str(train.at[idx, 'id']): train.at[idx, 'songs'] for idx in tqdm(train.index)}
date_dic = {idx: song_meta.at[idx, 'issue_date'] for idx in tqdm(song_meta.index)}

In [ ]:
def remove_seen(seen, items):
    seen = set(seen)
    return [i for i in items if i not in seen]

In [ ]:
def rec(df, base_df, df_mapped):
    # recommendation
    res = []
    for idx in tqdm(df.index):
        songs, tags, date, ID = df.at[idx, 'songs'], df.at[idx, 'tags'], df.at[idx, 'updt_date'], df.at[idx, 'id']
        try:
            most_id = [x[0] for x in train_mapped.similar_by_vector(df_mapped.get_vector(str(ID)), topn = 200)]
            get_songs, get_tags = [], []
            for i in most_id:
                get_songs += song_dic[i]
                get_tags += tag_dic[i]
            get_songs = list(pd.value_counts(get_songs)[: 200].index)
            get_tags = list(pd.value_counts(get_tags)[: 20].index)

            get_songs = remove_seen(songs, get_songs)[:100]
            get_tags = remove_seen(tags, get_tags)[: 10]

            if len(get_songs) < 100:
                get_songs += remove_seen(get_songs, base_df.at[idx, 'songs'])[: 100 - len(get_songs)]
            if len(get_tags) < 10:
                get_tags += remove_seen(get_tags, base_df.at[idx, 'tags'])[: 10 - len(get_tags)]

            res.append({
                'id': ID,
                'songs': get_songs,
                'tags': get_tags
            })
        except:
            res.append({
                'id': ID,
                'songs': base_df.at[idx, 'songs'],
                'tags': base_df.at[idx, 'tags']
            })
    res = pd.DataFrame(res)
    return res

In [ ]:
valid_res = rec(valid, valid_cold, valid_mapped)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [ ]:
valid_res[valid_res.songs.apply(len) != 100]

,id,songs,tags


In [ ]:
valid_res.to_json(base_path + 'submissions/valid_res.json', orient = 'records', force_ascii = False)

In [ ]:
test_res = rec(test, test_cold, test_mapped)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [ ]:
test_res[test_res.songs.apply(len) != 100]

,id,songs,tags


In [ ]:
test_res.to_json(base_path + 'submissions/test_res.json', orient = 'records', force_ascii = False)